Najbardziej bezpośrednią metodą znalezienia wartości własnych jest znalezienie miejsc zerowych wielomianu charakterystycznego. Natomiast wyzwaniem jest wyznaczenie tego wielomianu. Wynika to z faktu, że wyznaczenie tego np. z rozwinięcia Laplace'a (wykorzystującego minory) byłoby bardzo czasochłonne dla większych macierzy.

Alternatywną metodą pozwalającą na wyznaczenie współczynników wielomianu charakterystycznego jest metoda Faddeeva-LeVerriera. Jej wynikiem jest wektor współczynników $c_0, c_1, c_2, ..., c_N$ wielomianu charakterystycznego.
Wymaga ona wykonania $N-1$ kroków w trakcie których wykonywane jest mnożenie dwóch, pełnych macierzy oraz znalezienie _śladu_ macierzy $\text{tr}(B)$, będącego sumą elementów na głównej diagonali $B_{ii}$ (gdzie $i=1,2,...,N$).

Metoda Faddeeva-LeVerriera wykorzystuje formułę Jacobiego wiążącą wyznacznik macierzy ze śladem macierzy dołączonej. Warto zapoznać się też z twierdzeniem Cayleya-Hamiltona mówiącym, że każda macierz jest pierwiastkiem swojego wielomianu charakterystycznego (tj. podstawiając macierz jako element wielomianu otrzymamy macierz samych zer).

In [ ]:
using LinearAlgebra
import PolynomialRoots: roots

function FaddeevLeVerrier(A)
    n = size(A, 1)
    I = diagm(ones(n))
    B = copy(I)
    c = zeros(n + 1)
    k = 1
    c[n+1] = 1.0
    while k <= (n - 1)
        B = A * B
        c[n - k + 1] = -(1.0 / k) * tr(B)
        B += c[n - k + 1] * I
        k += 1
    end
    c[1] = -(1.0 / n) * tr(A, B)
    return c
end

Metoda Faddeeva-LeVerriera nie jest szybką metodą, ale istnieje ulepszenie nazwane metodą Preparata-Sarwate opisane m.in. przez _Fredrik Johansson_ w _On a fast and nearly division-free algorithm for the characteristic polynomial. 2020. ⟨hal-03016034v2⟩_

Nazywa się go też metodą małych i dużych kroków, ponieważ co $m$ małych kroków wykorzystujących przygotowane wcześniej potęgi macierzy i ich ślady, wykonywany jest duży krok.

In [ ]:
function PreparataSarwate(a)
    n = size(a, 1)
    m = floor(Int64, sqrt(n))
    A = [a]
    t = zeros(m)
    # precompute A^i and Tr(A^i)
    for i=1:m-1
        push!(A, A[1] * A[i])
        t[i] = tr(A[i])
    end
    t[m] = tr(A[m])

    c = zeros(n + 1)
    I = diagm(ones(n))
    B = copy(I)
    k = 1
    c[n + 1] = 1.0
    while k <= (n - 1)
        m = min(m, n - k)
        # compute coeffs.
        c[n - k - 0 + 1] = -(1.0 / k) * tr(A[1], B)
        for j = 1:m-1
            c[n - k - j + 1] = tr(A[j+1], B)
            for i = 0:j-1
                c[n - k - j + 1] += t[j - i] * c[n - k - i + 1]
            end
            c[n - k - j + 1] /= - (k + j)
        end
        # update B
        B = A[m] * B
        for j = 0:m-2
            B += c[n - k - j + 1] * A[m - j - 1]
        end
        for j = m-1
            B += c[n - k - j + 1] * I
        end

        k += m
    end
    c[1] = -(1.0 / n) * tr(A[1], B)
    return c
end

Ta metoda wymaga zdefiniowania operacji śladu iloczynu macierzy $\mathbf{A}$ i $\mathbf{B}$, który może być policzony bez faktycznego mnożenia tych macierzy.

W Julii wymaga to jedynie zdefiniowania nowej wersji metody `tr`.

In [ ]:
function LinearAlgebra.tr(A, B) # product trace: tr(A*B)
    n = size(A, 1)
    result = 0.0
    for i=1:n
        for j=1:n
            result += A[i,j]*B[j,i]
        end
    end
    return result
end

In [ ]:
A = randn(51, 51)
values = eigvals(A)
p = FaddeevLeVerrier(A)
λ = roots(p)

In [ ]:
using PyPlot
figure(dpi=150)
scatter(real.(λ), imag.(λ))
scatter(real.(values), imag.(values), marker=".")
xlabel("Re{λ}")
ylabel("Im{λ}")
legend(("Faddeev-LeVerrier", "Eigen"))

In [ ]:
A = randn(101, 101)
values = eigvals(A)
p = PreparataSarwate(A)
λ = roots(p)

In [ ]:
using PyPlot
figure(dpi=150)
scatter(real.(λ), imag.(λ))
scatter(real.(values), imag.(values), marker=".")
xlabel("Re{λ}")
ylabel("Im{λ}")
legend(("Preparata-Sarwate", "Eigen"))

In [ ]:
using BenchmarkTools

In [ ]:
function wlasne(A)
    p = FaddeevLeVerrier(A)
    λ = roots(p)    
end

@btime wlasne($A);

In [ ]:
function wlasne(A)
    p = PreparataSarwate(A)
    λ = roots(p)
end

@btime wlasne($A);

In [ ]:
@btime eigvals($A);

Niestety, w przypadku dużych macierzy nasz wielomian charakterystyczny jest wysokiego stopnia a jego współczynniki mogą mieć taką rozpiętość wartości, że trudno je objąć zakresem wartości Float64.

Poza tym, niezbyt często jesteśmy zainteresowani **wszystkimi** wartościami własnymi. Zwykle interesuje nas najwyżej kilka największych lub najmniejszych.